In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams.update({'font.size': 12})

In [ ]:
datasets = ["ugr16", "cidds", "ton"]
traces = dict()
traces_netshare = dict()
traces_stan = dict()
for d in datasets:
    traces[d] = pd.read_csv("data/raw/"+d+"/raw.csv")
    traces_netshare[d] = pd.read_csv("data/netshare/"+d+"/syn.csv")
    traces_stan[d] = pd.read_csv("data/stan/"+d+"/syn.csv")

In [ ]:
plt.figure(figsize=(12,3))
for i, d in enumerate(datasets):
    
    ports = traces[d]['dstport']
    ports_netshare = traces_netshare[d]['dstport']
    ports_stan = traces_stan[d]['dstport']
    
    ports = ports[ports < 1024]
    ports_netshare = ports_netshare[ports_netshare < 1024]
    ports_stan = ports_stan[ports_stan < 1024]
    
    ports = ports.value_counts() / len(traces[d])
    ports_netshare = ports_netshare.value_counts() / len(traces_netshare[d])
    ports_stan = ports_stan.value_counts() / len(traces_stan[d])
    
    ports = pd.DataFrame({"port":ports.index, "count":ports.values, "label":"real"})
    ports_netshare = pd.DataFrame({"port":ports_netshare.index, "count":ports_netshare.values, "label":"NetShare"})
    ports_stan = pd.DataFrame({"port":ports_stan.index, "count":ports_stan.values, "label":"STAN"})
    
    ports = ports[:5]
    for p in list(ports["port"]):
        if not p in list(ports_netshare["port"]):
            #print(p)
            ports_netshare = pd.concat([ports_netshare, pd.DataFrame({"port":p, "count":0, "label":"NetShare"}, index=[p])])
        if not p in list(ports_stan["port"]):
            ports_stan = pd.concat([ports_stan, pd.DataFrame({"port":p, "count":0, "label":"STAN"}, index=[p])])
     
    #print(ports)
    #print(ports_syn)
    #ports.index = ports["port"]
    ports_netshare.index = ports_netshare["port"]
    ports_stan.index = ports_stan["port"]
    
    ports_netshare = ports_netshare.loc[list(ports["port"])]
    ports_stan = ports_stan.loc[list(ports["port"])]
    
    all_ports = pd.concat([ports, ports_netshare, ports_stan])
    all_ports["port"] = all_ports["port"].astype("string")
    #all_ports = all_ports.reset_index(drop=True)
    #print(all_ports)
    plt.subplot(1,3,i+1)
    sns.barplot(data=all_ports, x="port", y="count", hue="label")
    plt.xlabel("")
    if i==0:
        plt.ylabel("Frequency")
    else:
        plt.ylabel("")
    plt.title(d.upper())
    #plt.show()
    
plt.suptitle("Top 5 service destination port numbers", y=0.0)
    